In [33]:
import pandas as pd
from time import strptime

Código de merge datasets environment y production y parseo timestamp (nuevas columnas -> datetime, hour (h,m,s))

# DATASETS ENVIROMENT + PRODUCTION

In [34]:
df_env = pd.read_csv('dataset/environment.csv')
df_prod = pd.read_csv('dataset/production.csv')

## Timestamp processing

In [35]:
df_env['tmstp'] = pd.to_datetime(df_env['tmstp'])
df_env.head(10)

,tmstp,station_id,station,Temp,Hum,WS,WD,Pre,Cond,Evt
0,2010-07-01 00:00:00,1,LPMA,19.0,68.0,11.10,NNE,1023.0,Partly Cloudy,NaN
1,2010-07-01 00:30:00,1,LPMA,19.0,68.0,9.30,NNE,1022.0,Partly Cloudy,NaN
2,2010-07-01 01:00:00,1,LPMA,18.5,70.0,3.64,Variable,1022.0,Scattered Clouds,NaN
3,2010-07-01 01:30:00,1,LPMA,19.0,78.0,3.70,Variable,1022.0,Partly Cloudy,NaN
4,2010-07-01 02:00:00,1,LPMA,19.0,83.0,5.59,North,1022.0,Partly Cloudy,NaN
5,2010-07-01 02:30:00,1,LPMA,19.0,83.0,13.00,NNE,1022.0,Partly Cloudy,NaN
6,2010-07-01 03:00:00,1,LPMA,19.0,73.0,14.80,NE,1021.0,Partly Cloudy,NaN
7,2010-07-01 03:30:00,1,LPMA,18.0,77.0,13.00,NE,1021.0,Partly Cloudy,NaN
8,2010-07-01 04:00:00,1,LPMA,19.0,68.0,9.30,NNE,1021.0,Partly Cloudy,NaN
9,2010-07-01 04:30:00,1,LPMA,19.0,73.0,11.10,NNE,1021.0,Partly Cloudy,NaN


In [36]:
df_prod['tmstp'] = pd.to_datetime(df_prod['tmstp'])
df_prod.head()

,id,tmstp,total,thermal,hydro,eolic,biomass,solar
0,1,2010-01-01 00:00:00,110.73,70.79,33.73,0.27,6.01,0.0
1,2,2010-01-01 00:15:00,110.71,70.77,33.49,0.39,6.13,0.0
2,3,2010-01-01 00:30:00,110.81,70.78,33.56,0.61,6.10,0.0
3,4,2010-01-01 00:45:00,110.31,70.83,32.91,0.75,6.05,0.0
4,5,2010-01-01 01:00:00,109.64,70.45,32.15,1.14,5.73,0.0


## Merge

In [37]:
df_merged = pd.merge(df_env, df_prod, on='tmstp')
df_merged['datetime'] = df_merged['tmstp'].dt.strftime('%Y-%m-%d')
df_merged['hour'] = df_merged['tmstp'].dt.strftime('%H:%M:%S')

df_merged.head()
df_merged.to_csv('dataset\env+prod.csv', index=False)

## Filtro Solar

In [38]:
df_env_prod = pd.read_csv('dataset/env+prod.csv')

In [39]:
solar_array = df_env_prod['solar'].unique() 
solar_array

array([0.   , 0.23 , 0.34 , ..., 0.163, 7.086, 3.075])

In [40]:
max_solar = df_env_prod[df_env_prod['solar'] > 16] #encontrar max
max_solar.head(50)

,tmstp,station_id,station,Temp,Hum,WS,WD,Pre,Cond,Evt,id,total,thermal,hydro,eolic,biomass,solar,datetime,hour
69337,2012-03-30 12:00:00,1,LPMA,21.0,38.0,13.00,Variable,1017.0,Partly Cloudy,NaN,78673,116.78,75.99,5.79,16.9,2.22,16.11,2012-03-30,12:00:00
69338,2012-03-30 12:00:00,1,LPMA,21.0,38.0,13.00,Variable,1017.0,Partly Cloudy,NaN,113809,116.78,75.99,5.79,16.9,2.22,16.11,2012-03-30,12:00:00
69339,2012-03-30 12:00:00,2,LPPS,19.0,88.0,25.89,NNW,1017.0,Scattered Clouds,NaN,78673,116.78,75.99,5.79,16.9,2.22,16.11,2012-03-30,12:00:00
69340,2012-03-30 12:00:00,2,LPPS,19.0,88.0,25.89,NNW,1017.0,Scattered Clouds,NaN,113809,116.78,75.99,5.79,16.9,2.22,16.11,2012-03-30,12:00:00


# HUE data (production calculator)

## Quitar comillas

In [1]:
import pandas as pd
import csv

df1 = pd.read_csv('dataset/pvwatts_hourly.csv')
df2 = pd.read_csv('dataset/pvwatts_monthly.csv')

for col in df1.columns:
    df1[col] = df1[col].astype(str).str.replace('"', '')
    
for col in df2.columns:
    df2[col] = df2[col].astype(str).str.replace('"', '')

# Guardar el dataset sin comillas
df1.to_csv('dataset/pvwatts_hourly.csv', index=False, quoting=csv.QUOTE_NONE)
df2.to_csv('dataset/pvwatts_monthly.csv', index=False, quoting=csv.QUOTE_NONE)

## Preprocessing timestamps

In [22]:
import pandas as pd
import csv
import glob as gl

path = gl.glob('dataset/pvwatts_hourly_*.csv')
dfs = []  

for file in path:
    dfs.append(pd.read_csv(file, low_memory=False))

df = pd.concat(dfs, ignore_index=True)  # df con todos los dfs

df.to_csv('dataset/new_pvwatts_hourly.csv', index=False, quoting=csv.QUOTE_NONE)

#esto para cada uno
# df = pd.read_csv('dataset/pvwatts_hourly_5.csv')
# df['timestamp'] = pd.to_datetime(df.assign(Year=2015)[['Year', 'Month', 'Day', 'Hour']])
# df.to_csv('dataset/pvwatts_hourly_5.csv', index=False, quoting=csv.QUOTE_NONE)
